# AI in Medicine: Data Science - Basics IV

## Python Programming: Deep Learning using *Keras*

*Instructor:* Moritz Seiler, AG Ritter, Charité - Universitätsmedizin Berlin (moritz.seiler@charite.de) <br>
*Target audience:* Medical students from Charité<br>
*Course date:* July 2020

**First, please make sure, that you're using the Python 3.6 kernel!**<br> You can check that in the top right corner of your browser window. If you're using a different Version, go to the Tab *Kernel* --> *Change Kernel* and select *Python 3.6*.

**TensorFlow Update:**<br>
The pre-installed version of the tensorflow in this environment is 1.1.0, but the following notebook requires version 2.0.0. Please update the tensorflow library by executing the following cell.

In [ ]:
!pip install --upgrade tensorflow

## 1. Aims of this session

This tutorial introduces **deep learning** in Python with ***Keras***. It is meant to be an **introduction to the *Keras* library** and **<ins>not</ins> a deep dive into the theory** surrounding deep learning. The goal of this session is that you will be able to implement your own neural network in Python.

## 2. Learning goals

### Theory

Deep Learning
- Definition
- Fully-connected Neural Networks
- Activation functions
- Forward pass
- Backward pass
- Convolutional Neural Networks

### Practical

1. Deep Learning Playground
2. Data set
3. Keras
    - Introduction
    - Workflow
        - Prepare data
        - Build model
        - Compile model
        - Fit model
        - Predict
        - Evaluate model
        - Save model


## 3. References

- Deep Learning<br>
https://www.deeplearningbook.org/<br>
https://cs231n.github.io/

- Keras<br>
https://keras.io/guides/<br>
https://www.tensorflow.org/tutorials/keras/classification

- Data<br>
https://www.kaggle.com/uciml/pima-indians-diabetes-database

## 4. Theory

See accompanying slide deck.

## 5. Practical

### 5.1 Playground excersises

The *Tensorflow* Playground is an interactive visualisation of neural networks in your browser. In this playground, you can choose different hyper-parameters of your neural network and visually observe the training process. This allows you to get some intuition on neural network workings

https://playground.tensorflow.org

### Exercises

**1.** Play around and get some intution!

**2.** Let's try a different dataset. Investigate the effects of the learning rate on the training results. <br>

**3.** We now select the spiral dataset and add noise to the training data (noise=50). What do you observe using the following network architecture?<br>
https://playground.tensorflow.org/#activation=sigmoid&batchSize=10&dataset=spiral&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=50&networkShape=2&seed=0.00906&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=true&ySquared=true&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false

### 5.2 Data set

The data we are using in this tutorial is the Diabetes data set that you have already prepared in the previous tutorial.

<ins>Source:</ins><br>
*Smith, J.W., Everhart, J.E., Dickson, W.C., Knowler, W.C., & Johannes, R.S. (1988). Using the ADAP learning algorithm to forecast the onset of diabetes mellitus. In Proceedings of the Symposium on Computer Applications and Medical Care (pp. 261--265). IEEE Computer Society Press.*


In [4]:
# Import the Pandas and NumPy library
import pandas as pd
import numpy as np

# Read the Diabetes data set 
diabetes = pd.read_csv('diabetes.csv')

In [ ]:
diabetes.head()

### 5.3 Keras


Keras is a high-level neural networks API, capable of running on top of Tensorflow, Theano, and CNTK. It enables fast experimentation through a high level, user-friendly, modular and extensible API

<img src="https://keras.io/img/logo.png" width="40%">

<ins>Source:</ins> https://keras.io/img/logo.png

### Installation

For an installation of the *Keras* library, you have to enter the following command in the terminal.

```python
!pip install tensorflow
```

Keras comes packaged with *TensorFlow 2.0* as `tensorflow.keras`. To start using *Keras*, simply install *TensorFlow 2.0*.

*Keras* is a model-level library, providing high-level building blocks for developing deep learning models. It doesn't handle itself low-level operations such as tensor products, convolutions and so on. Instead, it relies on a specialized, well-optimized tensor manipulation library to do so, serving as the "backend engine" of Keras. 

Rather than picking one single tensor library and making the implementation of *Keras* tied to that library, *Keras* handles the problem in a modular way, and several different backend engines can be plugged seamlessly into *Keras*.

### Importing the library

We aren't going to import the whole *Keras* library but only specific classes that we need for our implementation. This will later be done using the following command:

```python
import tensorflow
from tensorflow.keras.xyz import ...
```

### Workflow

Training your first simple neural network with *Keras* doesn't require a lot of code, but we're going to start slow, taking it step-by-step, ensuring you understand the process of how to train a network. The workflow in *Keras* consits of the following steps:
1. **Prepare data**
2. **Build model**
3. **Compile model**
4. **Fit model**
5. **Predict**
6. **Evaluate model**
7. **Save model**

We're now going to introduce all these steps in more detail in the following part of the notebook. 

#### 1. Data preparation

Before we start building our neural network, we have to prepare our data. A first step should always be data inspection, which you have already done in the last tutorial. Here, we will split our data in different subsets to train our model and will standardise the data.

**Data splitting**

As you have learnt in the last tutorial, we have to split our dataset in subsets since we do not know the (hidden) data generating distribution to calculate the true error of our model. For this, we split the diabetes dataset into a training set, validation set and test set using the following split 60-20-20 to approximate the true error. We use the `train_test_split()`function from them `sklearn.model_selection` module to perform the splitting.

Here, we split the dataset only in a training and test set. Later in the training step, we use *Keras*' functionality to create a validation set from the training set. 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(diabetes[['Glucose','BMI','Age']], diabetes['Outcome'], 
                                                    test_size=0.2)

Let's check how many observations are in each set.

In [ ]:
print("Number of training samples: %i" % len(X_train))
print("Number of test samples: %i" % len(X_test))

**Tranformations**

Convergence of neural networks is ususally faster if the **average of each input variable**  over the training set is **close to zero**. In general, any shift of the average input away from zero will bias the updates in a particular direction and thus slow down learning. Additionally, **scaling the inputs** - to have **about the same covariance** - speeds up learning because it helps to balance out the rate at which the weights connected to the input nodes learn. Therefore, we will *standardize* our data 

$$z=\frac{x-\mu}{\sigma} $$

with mean $\mu$ and standard deviation $\sigma$. This transformation will speed up and stabilise the training process. We can use the `StandardScaler()` function from the `sklearn.preprocessing` module to perform the standardisation.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Instatiate scaler
scaler = StandardScaler()

# Train scaler on the data set
scaler.fit(X_train)

# Transform datasets
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Let us have a look at our transformed training set.

In [ ]:
X_train[:10,:]

Let us also check the mean and standard deviation of our standardised training dataset.

In [ ]:
print(np.mean(X_train,0))
print(np.std(X_train,0))

#### 2. Model Building

The core data structures of *Keras* are **layers** and **models**. There are three ways to create Keras models:

- The Sequential model is simply a linear stack of layers. *(for simple implementations)*
- The Functional API fully-featured API that supports arbitrary model architectures. *(for more complex implementations)*
- Model subclassing, where you implement everything from scratch on your own. *(for more complex implementations)*

We are going to use the **Sequential model** class in this tutorial. You simply need to import the model from the *Keras* library and initialise a model object using the following commands:

In [ ]:
import tensorflow 
from tensorflow.keras.models import Sequential

model = Sequential()

Then, we can stack layers and add those to our model object. Before we do so, the implementation of the layer types  
- dense layer and 
- convolutional layer 

is introduced, which you are going to use later in this notebook. A complete list of supported layer types is given in the *Keras* documentation https://keras.io/api/layers/. First, we have to import these layers from the *Keras* layer module using the following command:

```python
from tensorflow.keras.layers import Dense, Conv2D
```

**Dense layer**<br>
A dense layer/fully-connected layer is a layer in which each input neuron is connected to each output neuron, the parameters units just tells you the dimensionality of your output. 

```python
Dense(units='dimensionality of the output space',
      activation='activation function to use',
      ...)
```
<ins>Example:</ins>

```python
Dense(units=6, activation='relu',...)
```

**Convolutional layer**<br>
The convolutional layer’s parameters consist of a set of learnable filters. During the forward pass, we convolve each filter across the width and height of the input and compute dot products between the entries of the filter and the input at any position. By doing so, we produce an activation map that gives the responses of that filter at every spatial position.

```python
Conv2D(filters='the dimensionality of the output space (i.e. the number of output filters in convolution)', 
       kernel_size='specifying the height and width of the 2D convolutional kernel', 
       strides=' specifying the strides of the convolution along the height and width', 
       activation='activation function to use',
       ...)
```
<ins>Example:</ins>

```python
Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), activation='relu',...)
# if width and height are equal you can write in short
Conv2D(filters=16, kernel_size=3, strides=1, activation='relu',...)
```

We can simply add these layers to the Sequential model using the `add()` method.
Here, we add  
- an dense input layer with 3 units, 
- a dense sigmoid output layer with 1 unit

to create a **fully-connected neural network**.

In [ ]:
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense

model.add(Input(shape=(3,)))
model.add(Dense(1, activation='sigmoid'))

We can display the content of our neural network by calling the `summary()` method.

In [ ]:
model.summary()

*Why does our network have 4 parameters?*

#### 3. Model compilation

In order to train our neural network, we need to compile our model. This requires us to define a loss function and an optimiser to do so. A list already implemented losses and optimisers can be found in the documentary:

- losses: https://keras.io/api/losses/,
- optimisers: https://keras.io/api/optimizers/. 

**Loss function**

The purpose of loss functions is to compute the quantity that a model should seek to minimize during training. 
This function will essentially calculate how poorly our model is performing by comparing what the model is predicting with the actual value it is supposed to output. Therefore, the selection of the loss function depends on the the task.

<ins>Examples:</ins><br>

*Classification:*
- `BinaryCrossentropy()`<br> Computes the cross-entropy loss between true labels and predicted labels *(binary)*.
- `CategoricalCrossentropy()`<br> Computes the cross-entropy loss between true labels and predicted labels *(multi-class)*.
- ...

*Regression:*
- `MeanSquaredError()`<br> Computes the mean of squares of errors between labels and predictions.
- ...

You can import the loss functions from the *Keras* losses module by entering the following command:

```python
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy, MeanSquaredError
```

**Optimiser**

We have already dealt with the loss function, which is a mathematical way of measuring how wrong your predictions are. In other words, the loss function lets us quantify the quality of any particular set of model parameters. The goal of optimisation is to find those parameteres that minimize the loss function. For this, we can compute the best direction along which we should change our parameters that is mathematically guaranteed to be the direction of the steepest descend. This direction will be related to the gradient of the loss function.

<ins>Examples:</ins>
- `SGD()`<br> Stochastic gradient descent is an implementation of gradient descent - that you know from the lecture - on batches of observations or single random examples on each pass.

- `Adam()`<br> Adam stands for adaptive moment estimation, and is another way of using past gradients to calculate current gradients. Adam also utilizes the concept of momentum by adding fractions of previous gradients to the current one.

You can import the optimiser from the `tensorflow.keras.optimizers` module by entering the following command:

```python
from tensorflow.keras.optimizers import SGD, Adam
```

We instatiate the loss function and the optimiser before we pass them to the model.

In [ ]:
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import SGD

loss = BinaryCrossentropy()

lr = 4e-2 #lr = 0.04
optimiser = SGD(learning_rate=lr)

model.compile(loss=loss,
              optimizer=optimiser, 
              metrics=['accuracy'])

After compiling our model, we're now ready to start the training.

#### 4. Model training

To train our model we can simply use the `fit()` method. Here, we just have to provide the *number of epochs*, the *batch size* and the *training dataset* (features and targerts). Instead of providing a separate validation dataset, we use the `validation_split` functionality, which uses the last x% observations of the training dataset before shuffling to create a validation set.

At the end of each epoch, the model will iterate over the validation dataset and compute the validation loss and validation metric.

The `fit()` method outputs a `History` object; its `History.history` attribute is a record of loss and metrics values (training + validation).

In our example, we use 50 *epochs* and a *batch size* of 64 to train our model. Additionally, we use a validation split of 0.25 such that our overall data split in training, validation and test set is 60-20-20.

In [ ]:
epochs = 50
batch_size = 64


history = model.fit(x=X_train, y=y_train,
                validation_split=0.25,
                epochs=epochs, 
                batch_size=batch_size)

The `fit()` methods prints the loss and metric values after every epochs, but we can also use the `History` object to visualise those values to get intuition about the performance of our model.

In [ ]:
import matplotlib.pyplot as plt

n_epochs = np.arange(0,epochs)

fig, (ax1,ax2) = plt.subplots(2,1,figsize=(8,16))
ax1.plot(n_epochs, history.history['loss'], label='training loss')
ax1.plot(n_epochs, history.history['val_loss'], label='validation loss')
ax1.set_ylim(-0.05,1.05)
ax1.legend()

ax2.plot(n_epochs, history.history['accuracy'], label='training accuracy')
ax2.plot(n_epochs, history.history['val_accuracy'], label='validation accuracy')
ax2.set_ylim(-0.05,1.05)
ax2.legend()

plt.show()

#### 5. Prediciton

After training, we can use our model to predict the target variable using the `predict()` method, which returns the probabilities $P(Outcome=1|X,w)$.

In [ ]:
y_pred = model.predict(X_test).ravel()

In [ ]:
y_pred

If you're not interested in the probability $P(Outcome=1|X,w)$ but simply want to predict the class label, you can use the `predict_classes()` method. This method uses a threshold of 0.5 to classify examples.

In [ ]:
model.predict_classes(X_test).ravel()

#### 6. Model evaluation

To see how good/bad those predictions are, we've to evaluate them using the built-in `evaluate()` method. This returns the loss and metrics values for the model in test mode.

In [ ]:
loss_and_metrics = model.evaluate(X_test, y_test, 
                                  batch_size=batch_size)

In addition to loss and metric values, we're going to use the receiver operating characteristics (ROC) and the area under the curve (AUC) to evaluate our model. We can use the `roc_curve()` and `auc` function from the `sklearn.metrics` module to calculate those values. 

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds_keras = roc_curve(y_test, y_pred)
auc_keras = auc(fpr, tpr)

After calculating all values, we can use *matplotllib* to plot the ROC curve.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,1,figsize=(12,8))
ax.plot([0, 1], [0, 1], 'k--')
ax.plot(fpr, tpr, label='Keras (area = {:.3f})'.format(auc_keras))
ax.set_xlabel('False positive rate')
ax.set_ylabel('True positive rate')
ax.set_title('ROC curve')
ax.legend(loc='best')

plt.show()

#### 7. Model saving

A Keras model consists of multiple components:

- An architecture, or configuration, which specifyies what layers the model contains, and how they're connected.
- A set of weights values (the "state of the model").
- An optimizer (defined by compiling the model).
- A set of losses and metrics (defined by compiling the model).

We can save these pieces to disk at once into a single archive in the *Tensorflow* `SavedModel`.

```python
model.save('path/to/location')
```

We can save our neural network to the working directory by entering the following command:

In [ ]:
model.save('my_nn')

This can then simply be loaded using the following command:

```python
model = tensorflow.keras.models.load_model('my_nn')
```

**Put everything together**<br>
After having itroduced all the steps separately in more detail, everything is put together in one cell, so you don't have to scroll too much.

In [ ]:
import tensorflow 
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import SGD

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc

import matplotlib.pyplot as plt


## Data preparation
# Data Splitting
X_train, X_test, y_train, y_test = train_test_split(diabetes[['Glucose','BMI','Age']], 
                                                    diabetes['Outcome'], 
                                                    test_size=0.2)

# Data transformation       
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)



## Model building
model = Sequential()
model.add(Input(shape=(3,)))
model.add(Dense(units=4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

## Model compilation
# Loss function
loss = BinaryCrossentropy()

# Optimiser
lr = 4e-2 
optimiser = SGD(learning_rate=lr)

model.compile(loss=loss,
              optimizer=optimiser, 
              metrics=['accuracy'])

## Model training
epochs = 50
batch_size = 16

history = model.fit(x=X_train, y=y_train, 
                    validation_split=0.25,
                    epochs=epochs,
                    batch_size=batch_size)


n_epochs = np.arange(0,epochs)

fig, (ax1,ax2) = plt.subplots(2,1,figsize=(8,16))
ax1.plot(n_epochs, history.history['loss'], label='training loss')
ax1.plot(n_epochs, history.history['val_loss'], label='validation loss')
ax1.set_ylim(-0.05,1.05)
ax1.legend()

ax2.plot(n_epochs, history.history['accuracy'], label='training accuracy')
ax2.plot(n_epochs, history.history['val_accuracy'], label='validation accuracy')
ax2.set_ylim(-0.05,1.05)
ax2.legend()
plt.show()

## Prediciton
y_pred = model.predict(X_test).ravel()

## Model evaluation
loss_and_metrics = model.evaluate(X_test, y_test, 
                                  batch_size=batch_size)

fpr, tpr, thresholds_keras = roc_curve(y_test, y_pred)
auc_keras = auc(fpr, tpr)

fig, ax = plt.subplots(1,1,figsize=(12,8))
ax.plot([0, 1], [0, 1], 'k--')
ax.plot(fpr, tpr, label='Keras (area = {:.3f})'.format(auc_keras))
ax.set_xlabel('False positive rate')
ax.set_ylabel('True positive rate')
ax.set_title('ROC curve')
ax.legend(loc='best')

plt.show()

## Save model
model.save('my_nn')

## 6. Discussion

In this notebook, the implementation of a neural network using the *Keras* library was introduced.
The workflow to build and train a neural network was presented using a step-by-step introduction.
- **Data Preparation:** We splitted the dataset and standardised the input features.
- **Model Building:** We created a neural network using the Sequential model class and introduced dense and convolutional layers.
- **Model compilaton:** We compiled a neural network and introduced different loss functions and optimisers.
- **Model training:** We trained a neural network and visualised the training process.
- **Model prediction and evaluation:** We used the trained network to predict the target variable and evaluated the network using different metrics.
- **Save model:** We saved all model components in a single archive `SavedModel`

## 7. Exercises

Now that you know the basics to implement a neural network in *Keras*, it's time for you to get your hands dirty and get started.

*Tip:* The following resources might be helpful:
- www.stackoverflow.com
- Keras documentation: https://keras.io/api/

### 7.1 Fully-connected neural network (Basic)

<ins>**Task:**</ins> Please **implement your own classifier to predict the outcome classes** in the diabetes dataset. <br>
<ins>**Model:**</ins> Use a **fully-connected neural network** with at least 1 hidden layer.<br> (*Note:* Since this is still a binary classification task, please use 1 unit and the sigmoid activation in your output layer.)<br>
<ins>**Data:**</ins> Use all available data features to predict the outcome class.

*Do your results look different from the ones in the example above?*

#### Data preparation

In [ ]:
### Data preparation ###
# TO DO: Split and transform dataset




#### Model building

In [ ]:
### Model building ###
model = Sequential()
# TO DO: Create a neural network with at least 1 hidden layer



model.summary()

#### Model compilation

In [ ]:
# TO DO: Define loss function and optimiser


# TO DO: compile model




#### Model training

In [ ]:
# TO DO: Choose the training length and batch size

history = # TO DO: Train your model (Don't forget validation_split)



In [ ]:
n_epochs = np.arange(0,epochs)

fig, (ax1,ax2) = plt.subplots(2,1,figsize=(8,16))
ax1.plot(n_epochs, history.history['loss'], label='training loss')
ax1.plot(n_epochs, history.history['val_loss'], label='validation loss')
ax1.set_ylim(-0.05,1.05)
ax1.legend()

ax2.plot(n_epochs, history.history['accuracy'], label='training accuracy')
ax2.plot(n_epochs, history.history['val_accuracy'], label='validation accuracy')
ax2.set_ylim(-0.05,1.05)
ax2.legend()
plt.show()

#### Model evaluation

In [ ]:
# TO DO: Calculate test loss and accurcy

### 7.2 Convolutional neural network (Advanced)

The second task is to implement a multi-class image classifier for the famous MNIST dataset. This dataset contains 70,000 (28*28 pixel) images of digits 0-9.

<img src="https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png" width="40%">

*Source: https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png*


<ins>**Task:**</ins> Please **implement your own multi-class image classifier to predict the outcome classes** in the diabetes dataset. <br>
<ins>**Model:**</ins> Use a **convolutional neural network** with at least 1 hidden convolutional layer.<br> (*Note:* Since this is still a multi-class classification task, please use 10 units and the softmax activation in your output layer.)<br>
<ins>**Data:**</ins> Use the 28*28 pixel images to predict the digit class.


#### Data preparation

Before we can use the MNIST data to predict the class labels, we have to **split** the dataset, **normalise** (transform range from 0-255 to 0-1) and reshape the data and **one-hot encode** the target. The data preparation step is already done for you, just execute the following cell and check the data.

In [ ]:
# Import the MNIST dataset
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load MNIST images and labels and normalise (range 0 to 1) image data
(X_train, y_train), (X_test, y_test) = mnist.load_data() # 60,000 training + 10,000 test images/labels
X_train = X_train / 255.0
X_test  = X_test / 255.0

# Reshape dataset to have a single channel 
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)) # The CNN requires this layout (batch_size, height, width, n_channels)
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)) # The CNN requires this layout (batch_size, height, width, n_channels)

# One-hot encode target values
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Let's have a look at the image data...

In [ ]:
fig, (ax1,ax2,ax3,ax4) = plt.subplots(1,4, figsize=(12,4))
ax1.imshow(X_train[0,:,:,0])
ax2.imshow(X_train[1,:,:,0])
ax3.imshow(X_train[2,:,:,0])
ax4.imshow(X_train[3,:,:,0]);

and the one-hot encoded target variable.

In [ ]:
y_train[:4]

Instead of the categorical class values, we now have a vector of length equal to the number of categories. The i-th component is equal to 1 if target is assigned to the i-th category and all other components are 0. By using this transformation, we transform the categorical variable into a numerical form.

#### Model building

Now, it's time to implement your convolutional neural network. The last layers of the network are fully-connected layers with 256 units and ReLu activation function as well as a fully-connected output layer with 10 units and softmax activation function, since we have a classification task with 10 classes.

In [ ]:
from tensorflow.keras.layers import Conv2D, Flatten

model = Sequential()
model.add(Input(shape=(28,28,1)))
# TO DO: Add at least 1 convolutional layer

model.add(Flatten()) # This flattens your 28*28 image into a vector of length 28*28= 784. 
model.add(Dense(256, activation='relu')) 
model.add(Dense(10, activation='softmax')) 

model.summary()

#### Model compilation

In [ ]:
from tensorflow.keras.losses import CategoricalCrossentropy

loss = CategoricalCrossentropy()
# TO DO: Define loss function

# TO DO: Compile your model

#### Model training

In [ ]:
# TO DO: Choose the training length and batch size

history = # TO DO: Train your model (Don't forget validation_split!)


In [ ]:
n_epochs = np.arange(0,20)

fig, (ax1,ax2) = plt.subplots(2,1,figsize=(8,16))
ax1.plot(n_epochs, history.history['loss'], label='training loss')
ax1.plot(n_epochs, history.history['val_loss'], label='validation loss')
ax1.set_ylim(-0.05,1.05)
ax1.legend()

ax2.plot(n_epochs, history.history['accuracy'], label='training accuracy')
ax2.plot(n_epochs, history.history['val_accuracy'], label='validation accuracy')
ax2.set_ylim(-0.05,1.05)
ax2.legend()

plt.show()

#### Model evaluation

In [ ]:
# TO DO: Calculate test loss and accurcy